In [18]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [19]:
marathon_counts = []

results_response = requests.get("https://www.marathonguide.com/results/")
results_html = results_response.content.decode("utf-8")
results_html_parsed = BeautifulSoup(results_html)

for year in range(2005,2023):
    result_row = results_html_parsed.body.find('a', attrs={'href': f'browse.cfm?Year={year}'})
    raw_count = result_row.next.next
    clean_count = raw_count.replace(" (all ", "").replace(")", "")

    year_count = {
        "year": year,
        "marathon_count": clean_count
    }
    
    marathon_counts.append(year_count)

marathon_count_df = pd.DataFrame(marathon_counts)

marathon_count_df.to_csv("data_input/marathon_counts_by_year.csv")


In [65]:
def get_participant_count(race_url):
    race_results_url = f'https://www.marathonguide.com/results/{race_url}'
    race_results_response = requests.get(race_results_url)
    race_results_html = race_results_response.content.decode("utf-8")
    race_results_parsed = BeautifulSoup(race_results_html)
    
    participant_count_element = race_results_parsed.body.find('input', attrs={'name': 'Max'})
    try:
        participant_count = int(participant_count_element.attrs['value'])
    except:
        participant_count = 0
    finally:
        return participant_count

In [66]:
## WARNING - This code takes just over 2 hours to run!!!

marathon_participants = []
used_marathon_urls = []

for year in range(2005,2023):
    year_result_url = f"https://www.marathonguide.com/results/browse.cfm?Year={year}"
    year_result_response = requests.get(year_result_url)
    year_result_html = year_result_response.content.decode("utf-8")
    year_result_html_parsed = BeautifulSoup(year_result_html)
    
    result_rows = year_result_html_parsed.find_all('a')
    for result_row in result_rows:
        if ('browse.cfm?MIDD=' in str(result_row.get('href'))):
            result_url = result_row.get('href')
            if result_url not in used_marathon_urls:
                used_marathon_urls.append(result_url)
                participant_count = get_participant_count(result_url)
                if participant_count > 0:
                    race_data = {
                        "year": year,
                        "marathon_name": result_row.text,
                        "participant_count": participant_count
                    }

                    print(race_data)
                    marathon_participants.append(race_data)
            
marathon_participants_df = pd.DataFrame(marathon_participants)

marathon_participants_df.to_csv("data_input/marathon_participants_by_race_by_year.csv")


{'year': 2005, 'marathon_name': 'Last Chance Marathon', 'participant_count': 22}
{'year': 2005, 'marathon_name': 'Christmas Marathon', 'participant_count': 77}
{'year': 2005, 'marathon_name': 'Jacksonville Marathon', 'participant_count': 827}
{'year': 2005, 'marathon_name': 'Dallas White Rock Marathon', 'participant_count': 3076}
{'year': 2005, 'marathon_name': 'Honolulu Marathon', 'participant_count': 24219}
{'year': 2005, 'marathon_name': 'Otter Creek Trail Marathon', 'participant_count': 31}
{'year': 2005, 'marathon_name': 'Kiawah Island Marathon', 'participant_count': 1021}
{'year': 2005, 'marathon_name': 'Rocket City Marathon', 'participant_count': 812}
{'year': 2005, 'marathon_name': 'Roxbury Marathon', 'participant_count': 12}
{'year': 2005, 'marathon_name': 'California International Marathon', 'participant_count': 3249}
{'year': 2005, 'marathon_name': 'Las Vegas Marathon', 'participant_count': 8186}
{'year': 2005, 'marathon_name': 'Luton Marathon', 'participant_count': 426}
{'y